In [14]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import time

In [15]:
def extract_by_value(data, column, value):
    """ Return the data with given value in the specified column
    """
    if not column in data.columns:
        raise ValueError('The column name is incorrect!')
    return data[data[column] == value]

In [16]:
def extract_asset_name_and_codes(data):
    """ Return a DataFrame including all assetName to assetCode correspondences from the input data
    """
    assetCode = "assetCode"
    if "assetCodes" in data.columns.tolist():
        assetCode = "assetCodes"
    subdata = data[['assetName', assetCode]]
    # Group by assetName
    return subdata.groupby('assetName')[assetCode].apply(set).reset_index()

In [17]:
def generate_split_dates(start_year, end_year):
    """ Generate a list of start & end dates for splitting the data (e.g. [('2016-01-01','2016-12-31')])
    """
    split_dates = []
    for year in range(start_year, end_year+1):
        split_dates.append((str(year)+'-01-01', str(year)+'-12-31'))
    return split_dates

In [18]:
def split_data_by_date(data, split_dates):
    """ Split the data by the input list of dates
    data: the input DataFrame
    split_dates: a list of start & end dates for splitting the data (e.g. [('2016-01-01','2016-12-31')])
    """
    data_split = {}
    for start_date, end_date in split_dates:
        condition1 = data['time'] >= pd.to_datetime(start_date).date()
        condition2 = data['time'] <= pd.to_datetime(end_date).date()
        data_split[start_date] = data[condition1 & condition2]
    return data_split

In [ ]:
def find_asset_name_map(market_train, news_train):
    """ Find assetName correspondences between market_train and news_train
    """
    # Get assetCode and assetName for both market_train and news_train data
    code_name_market = market_train[['assetCode', 'assetName']].drop_duplicates(subset=['assetCode'])
    code_name_news = news_train[['assetCodes', 'assetName']].drop_duplicates(subset=['assetCodes', 'assetName'])
    # Split "assetCodes"
    code_name_news['assetCodes'] = code_name_news['assetCodes'].str.strip('{}').str.split(',')
    # For each assetCode in the list of "assetCodes", prepare a new row for each assetcode
    assetCode = code_name_news.apply(lambda x: pd.Series(x['assetCodes']), axis=1).stack().reset_index(level=1, drop=True)
    assetCode.name = 'assetCode'
    code_name_news.drop('assetCodes', axis=1, inplace=True)
    code_name_news_joined = code_name_news.join(assetCode).reset_index(drop=True)
    code_name_news_joined['assetCode'] = code_name_news_joined['assetCode'].apply(lambda x: x.replace("'",""))
    code_name_news_joined['assetCode'] = code_name_news_joined['assetCode'].apply(lambda x: x.replace(" ",""))
    # Rename assetName to assetName_news
    code_name_news_joined.rename(columns={'assetName': 'assetName_news'}, inplace=True)
    # Merge two dataframes together
    code_name_merged = pd.merge(left=code_name_market, right=code_name_news_joined, how='left', on=['assetCode'])
    # Filter out Unknown and NAN assetName
    code_name_merged.dropna(inplace=True)
    condition = code_name_merged['assetName'] == 'Unknown'
    code_name_filtered = code_name_merged[~condition]
    # Build up a dictionary to establish assetName mapping from market_train to news_train
    assetName_map = {}
    # Note: iterrows can be very inefficient!
    for index, row in code_name_filtered.iterrows():
        if row['assetName'] != row['assetName_news']:
            assetName_map[row['assetName_news']] = row['assetName']
    return assetName_map

In [ ]:
def merge_data(market_train, news_train):
    """ Return the combined data by merging market_train and news_train on "time" and "assetCode"
    """
    """ Pre-process of market_train """
    # Convert "time" to datetime format (Note: Currently, we only keep the time to date)
    market_train['time'] = pd.to_datetime(market_train['time']).apply(lambda x: x.date())
    print('Convert time to datetime format is done for market data!')
    
    """ Pre-process of news_train """
    # Convert "time" to datetime format (Note: Currently, we only keep the time to date)
    news_train['time'] = pd.to_datetime(news_train['time']).apply(lambda x: x.date())
    print('Convert time to datetime format is done for news data!')
    
    # Get rid of some columns in news data (the list of dropped columns can be modified)
    drop_list = ['sourceTimestamp','firstCreated','sourceId','headline',
                 'takeSequence','provider','firstMentionSentence',
                 'sentenceCount','bodySize','headlineTag','marketCommentary',
                 'subjects','audiences','sentimentClass','urgency',
                 'wordCount','sentimentWordCount']
    news_train.drop(drop_list, axis=1, inplace=True)
    print('Drop columns is done for news data!')
    
    # Find assetName map from market_train to news_train
    assetName_map = find_asset_name_map(market_train, news_train)
    print('Find the assetName correspondences between market and news data!')
    
    # Adjust 'time' for news_train
    # First, get all the unique dates from news data and market data
    time_market = pd.DataFrame(market_train['time'].unique(), columns={'time'})
    time_news = pd.DataFrame(news_train['time'].unique(), columns={'time'})
    # Keep a copy of market date before merging
    time_market['time_market'] = time_market['time']
    # Merge the two dataframes,the merged dataframe should have the same length with time_news
    # Also fill the next trading date
    time_adjusted= pd.merge(left=time_market, right= time_news, how='right', on=['time'], sort=True).fillna(method='bfill')
    # Merge adjusted time to news data
    news_train_adjusted = pd.merge(left=news_train, right=time_adjusted, how='left', on=['time'], copy=False)
    del news_train
    # Modify 'time_market' as the new 'time' column
    news_train_adjusted.drop(['time'], axis=1, inplace=True)
    news_train_adjusted.rename(columns={'time_market': 'time'}, inplace=True)
    print('Adjust date is done for news data!')
    
    # Split market_train and news_train by year
    market_train_years = pd.to_datetime(market_train['time']).dt.year.unique()
    news_train_years = pd.to_datetime(news_train_adjusted['time']).dt.year.unique()
    start_year = min(np.amin(market_train_years), np.amin(news_train_years))
    end_year = max(np.amax(market_train_years), np.amax(news_train_years))
    print('Split data from year-%d to year-%d' % (start_year, end_year))
    split_dates = generate_split_dates(start_year, end_year)
    market_train_split = split_data_by_date(market_train, split_dates)
    del market_train
    print('Split market data is done!')
    news_train_split = split_data_by_date(news_train_adjusted, split_dates)
    del news_train_adjusted
    print('Split news data is done!')
    
    # Iterate over split market and news data
    if len(market_train_split.items()) != len(news_train_split.items()):
        raise ValueError('The split train and news data must have the same length!')  
    merged_data = pd.DataFrame([])
    for start_date, end_date in split_dates:
        print('Merge data from %s to %s ...' % (start_date, end_date))
        market_train_to_process = market_train_split[start_date]
        news_train_to_process = news_train_split[start_date]
        # Modify assetName in news data according to assetName_map
        news_train_to_process['assetName'] = news_train_to_process['assetName'].apply(lambda x: assetName_map[x] if x in assetName_map.keys() else x)
        news_train_to_process.drop(['assetCodes'], axis=1, inplace=True)
        """
        # Split "assetCodes"
        news_train_to_process['assetCodes'] = news_train_to_process['assetCodes'].str.strip('{}').str.split(',')
        # For each assetCode in the list of "assetCodes", prepare a new row
        assetCode = news_train_to_process.apply(lambda x: pd.Series(x['assetCodes']), axis=1).stack().reset_index(level=1, drop=True)
        assetCode.name = 'assetCode'
        # Drop 'assetCodes' column and join the new column of "assetCode" to news_train
        news_train_to_process.drop('assetCodes', axis=1, inplace=True)
        news_train_dropped = news_train_to_process.join(assetCode).reset_index(drop=True)
        news_train_dropped['assetCode'] = news_train_dropped['assetCode'].apply(lambda x: x.replace("'",""))
        news_train_dropped['assetCode'] = news_train_dropped['assetCode'].apply(lambda x: x.replace(" ",""))
        del news_train_to_process
        """
        # Group news_train by "time" and "assetCode" and then compute mean on each group
        news_train_grouped = news_train_to_process.groupby(['time','assetName'], sort=False).aggregate(np.mean).reset_index()
        del news_train_to_process
        # Merge two DataFrames
        merged_data = pd.concat([merged_data, 
                                 pd.merge(left=market_train_to_process, right= news_train_grouped, how='left', on=['time', 'assetName'], copy=False)],
                                 ignore_index=True)
        #merged_data.info()
        del market_train_to_process
        del news_train_grouped
    del market_train_split
    del news_train_split
    # Concatenate and return all DataFrames
    return merged_data

In [21]:
# Load market and news training data
market_train = pd.read_csv("./market_train_df.csv")
news_train = pd.read_csv("./news_train_df_1.csv")

In [156]:
# Show the assetName and assetCode correspondences in market_train
extract_asset_name_and_codes(market_train)

,assetName,assetCode
0,21Vianet Group Inc,{VNET.O}
1,2U Inc,{TWOU.O}
2,3Com Corp,{COMS.O}
3,3D Systems Corp,"{TDSC.O, DDD.N}"
4,3M Co,{MMM.N}
5,500.Com Ltd,{WBAI.N}
6,58.com Inc,{WUBA.N}
7,7 Days Group Holdings Ltd,{SVN.N}
8,8x8 Inc,{EGHT.O}
9,99 Cents Only Stores,{NDN.N}


In [157]:
# Get merged DataFrame
start_time = time.time()
merged_data = merge_data(market_train, news_train)
print("--- %s seconds ---" % (time.time() - start_time))

Convert time to datetime format is done for market data!
Convert time to datetime format is done for news data!
Drop columns is done for news data!
Split market data is done!
Split news data is done!
Merge data from 2007-01-01 to 2007-12-31 ...
Adjust date is done for news data!
Split assetCodes is done for news data!
Merge data from 2008-01-01 to 2008-12-31 ...
Adjust date is done for news data!
Split assetCodes is done for news data!
Merge data from 2009-01-01 to 2009-12-31 ...
Adjust date is done for news data!
Split assetCodes is done for news data!
Merge data from 2010-01-01 to 2010-12-31 ...
Adjust date is done for news data!
Split assetCodes is done for news data!
Merge data from 2011-01-01 to 2011-12-31 ...
Adjust date is done for news data!
Split assetCodes is done for news data!
Merge data from 2012-01-01 to 2012-12-31 ...
Adjust date is done for news data!
Split assetCodes is done for news data!
Merge data from 2013-01-01 to 2013-12-31 ...
Adjust date is done for news data!


In [158]:
merged_data.head()

,time,assetCode,assetName,volume,close,open,returnsClosePrevRaw1,returnsOpenPrevRaw1,returnsClosePrevMktres1,returnsOpenPrevMktres1,...,noveltyCount12H,noveltyCount24H,noveltyCount3D,noveltyCount5D,noveltyCount7D,volumeCounts12H,volumeCounts24H,volumeCounts3D,volumeCounts5D,volumeCounts7D
0,2007-02-01,A.N,Agilent Technologies Inc,2606900.0,32.19,32.17,0.005938,0.005312,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2007-02-01,AAI.N,AirTran Holdings Inc,2051600.0,11.12,11.08,0.004517,-0.007168,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2007-02-01,AAP.N,Advance Auto Parts Inc,1164800.0,37.51,37.99,-0.011594,0.025648,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2007-02-01,AAPL.O,Apple Inc,23747329.0,84.74,86.23,-0.011548,0.016324,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2007-02-01,ABB.N,ABB Ltd,1208600.0,18.02,18.01,0.011791,0.025043,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [159]:
# Extract all records for 'Apple Inc' from the merged table
extract_by_value(merged_data, 'assetCode', 'AAPL.O')

,time,assetCode,assetName,volume,close,open,returnsClosePrevRaw1,returnsOpenPrevRaw1,returnsClosePrevMktres1,returnsOpenPrevMktres1,...,noveltyCount12H,noveltyCount24H,noveltyCount3D,noveltyCount5D,noveltyCount7D,volumeCounts12H,volumeCounts24H,volumeCounts3D,volumeCounts5D,volumeCounts7D
3,2007-02-01,AAPL.O,Apple Inc,23747329.0,84.7400,86.230,-0.011548,0.016324,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1421,2007-02-02,AAPL.O,Apple Inc,22212416.0,84.7500,84.120,0.000118,-0.024469,-0.001091,-0.026983,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2844,2007-02-05,AAPL.O,Apple Inc,20737742.0,83.9400,84.300,-0.009558,0.002140,-0.008843,0.001477,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4272,2007-02-06,AAPL.O,Apple Inc,30893975.0,84.1500,84.450,0.002502,0.001779,0.002006,0.002135,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5702,2007-02-07,AAPL.O,Apple Inc,38180972.0,86.1500,84.495,0.023767,0.000533,0.022779,0.000392,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7134,2007-02-08,AAPL.O,Apple Inc,24264750.0,86.1800,85.430,0.000348,0.011066,0.001591,0.010226,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8562,2007-02-09,AAPL.O,Apple Inc,30756328.0,83.2700,85.880,-0.033767,0.005267,-0.024615,0.006735,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9994,2007-02-12,AAPL.O,Apple Inc,25873256.0,84.8800,84.440,0.019335,-0.016768,0.023276,-0.009432,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11428,2007-02-13,AAPL.O,Apple Inc,21250545.0,84.6300,85.160,-0.002945,0.008527,-0.012278,0.012093,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12860,2007-02-14,AAPL.O,Apple Inc,18150679.0,85.3000,84.630,0.007917,-0.006224,-0.001869,-0.012470,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [163]:
merged_data[merged_data['sentimentNeutral']>0]

,time,assetCode,assetName,volume,close,open,returnsClosePrevRaw1,returnsOpenPrevRaw1,returnsClosePrevMktres1,returnsOpenPrevMktres1,...,noveltyCount12H,noveltyCount24H,noveltyCount3D,noveltyCount5D,noveltyCount7D,volumeCounts12H,volumeCounts24H,volumeCounts3D,volumeCounts5D,volumeCounts7D
